# Starting 3Di Simulation Demo

## Install and Import required packages:

In [ ]:
!pip install datetime
!pip install threedi_api_client
!pip install threedi_scenario_downloader

In [5]:
import arcpy

In [22]:
from datetime import datetime, timedelta
import datetime
from getpass import getpass
import pandas as pd
import json
from threedi_api_client.threedi_api_client import ThreediApiClient
from threedi_api_client.api import ThreediApi
from threedi_api_client.versions import V3Api
from threedi_api_client.openapi import ApiException
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import numpy as np
import configparser
import os
import math
from time import sleep
from threedi_scenario_downloader import downloader as dl
from pathlib import Path

## Provide credentials and check connection 

In [7]:
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = "Insert your api key"


config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

In [ ]:
try:
    user = api_client.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

## Find and select model

In [ ]:
models = api_client.threedimodels_list(limit=5)  # limit to the first 5 results
for model in models.results:
    print(f"{model.name}")

In [ ]:
my_model = api_client.threedimodels_list(name__icontains='assen_pittelo - pittelo_klimaatsom (1) #19')
print(my_model)

my_model = my_model.results[0]

## Find organization ID to run the simulation under

In [ ]:
organisations = api_client.organisations_list()
for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

## Create Simulation and add event

In [ ]:
simulation_templates = api_client.simulation_templates_list(simulation__threedimodel__id=my_model.id)

for simulation_template in simulation_templates.results:
    print(simulation_template.id, simulation_template.name)

assert simulation_templates.count > 0, f"No simulation templates found for threedimodel {my_model.name}"

simulation_template_id = simulation_templates.results[0].id

simulation_name = input('Name your simulation\n')

my_simulation = api_client.simulations_from_template(
    data={
        "template": simulation_template_id,
        "name": simulation_name,
        "organisation": '04b793908f7c4b599c578c68203a2f0d',
        "start_datetime": datetime(2020, 11, 15, 14, 0),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

## Add Events

In [ ]:
#Add rain timeseries event
rain = api_client.simulations_events_rain_timeseries_create(
    simulation_pk=my_simulation.id, data={
        'offset':0,
        'values': [[0, 0.005], [900, 0.002], [1800, 0.001], [2700, 0.0005], [3600, 0]],
        'units': 'm/s'}
)


events = api_client.simulations_events(my_simulation.id)
print(events)

## Start and check status

In [ ]:
status = api_client.simulations_status_list(my_simulation.id)
print(status)

In [ ]:
api_client.simulations_actions_create(my_simulation.id, data={"name": "start"})

In [ ]:
while True:
    simulation_status = api_client.simulations_status_list(my_simulation.id)
    
    print(f"Simulation status: {simulation_status.name}")
    
    if simulation_status.name.lower() == 'finished':
        print("Simulation has finished.")
        break
    
    time.sleep(10)  # Wait for 10 seconds before the next iteration


# Using Lizard to simulate and download

## Post Processing

In [ ]:
scenario_name= input('Scenario name: ')
testdict = {
    "scenario_name": scenario_name,
    "process_basic_results": True,
}


api_client.simulations_results_post_processing_lizard_basic_create(simulation_pk=my_simulation.id, data = testdict)

api_client.simulations_results_post_processing_lizard_start_create(simulation_pk = my_simulation.id, data = testdict)
api_client.simulations_results_post_processing_lizard_status_list(simulation_pk=my_simulation.id)

In [ ]:
while True:
    scenario_status = api_client.simulations_results_post_processing_lizard_status_list(simulation_pk=my_simulation.id)
    
    print(f"Simulation status: {scenario_status.status}")
    
    if scenario_status.status.lower() == 'archived':
        print("Scenario is archived.")
        break
    
    time.sleep(10)  # Wait for 10 seconds before the next iteration

## Set Parameters

In [ ]:
#Provide login credentials
lizard_username = '__key__'
api_key = "Insert lizard api key"

dl.set_api_key(api_key)

#Enter scenario information
srs = "EPSG:28992"
raster_resolution = 1
name = "EsriRainfall" #scenario_name

## Download Max Waterlevel

In [ ]:
base = os.path.normpath(arcpy.env.workspace + os.sep + os.pardir)

base_dir = base
filename = r'water_lvl.tif'

out_path = os.path.join(base_dir, filename)
print(out_path)


scenarios = dl.find_scenarios_by_name(scenario_name, limit=1)
print(scenarios)
uuid = ' '
for d in scenarios:
    uuid = d['uuid']
    print(d['uuid'])

print(uuid)

dl.download_maximum_waterlevel_raster(uuid,srs,pathname = out_path)

## Add to scene (Optional)

In [ ]:
aprx = arcpy.mp.ArcGISProject('current')
print(aprx.listMaps()[0].name)
my_map = aprx.listMaps()[0]
my_map.addDataFromPath(out_path)

## Multi-Raster Download

In [ ]:

output_directory = os.path.normpath(arcpy.env.workspace + os.sep + os.pardir)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

start_time = datetime.datetime(2000, 1, 1, 15, 0)
time_interval = datetime.timedelta(minutes=1)
end_time = datetime.datetime(2000, 1, 1, 15, 15)

while start_time < end_time:
    formatted_time_filename = start_time.strftime("%Y%m%d%H%M")  # For creating filename
    formatted_time_request = start_time.strftime("%Y-%m-%dT%H:%M")  # For request
    print(formatted_time_request)
    
    output_filename = f"water_depth_{formatted_time_filename}.tif"
    full_output_path = os.path.join(output_directory, output_filename)
    
    try:
        dl.download_waterdepth_raster(uuid, srs, 1, formatted_time_request, pathname=full_output_path)
        print(formatted_time_request + " printed")
    except Exception as e:
        print(f"Error downloading raster for {formatted_time_request}: {str(e)}")
    
    start_time += time_interval